In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import genjax

In [3]:
from tracking.model import model_factory

In [26]:
model = model_factory(
    max_T = 10,
    width = 240,
    height = 320,
    pose_kernel_params = (1.0, 1.0)
)

In [27]:
key = jax.random.PRNGKey(1)
trace = model.simulate(key, (4,))

In [28]:
trace.get_retval()

(Array([[ 9.6480873e+01,  2.1060333e+02,  5.4421717e-01],
        [ 9.4966339e+01,  2.1043172e+02,  2.7124362e+00],
        [ 9.5183731e+01,  2.1158604e+02, -2.8669615e+00],
        [ 9.5091530e+01,  2.1141847e+02,  1.3867779e+00],
        [ 9.3629852e+01,  2.1206841e+02,  2.9440856e-01],
        [ 9.3335396e+01,  2.1236569e+02,  8.4228814e-02],
        [ 9.3335396e+01,  2.1236569e+02,  8.4228814e-02],
        [ 9.3335396e+01,  2.1236569e+02,  8.4228814e-02],
        [ 9.3335396e+01,  2.1236569e+02,  8.4228814e-02],
        [ 9.3335396e+01,  2.1236569e+02,  8.4228814e-02],
        [ 9.3335396e+01,  2.1236569e+02,  8.4228814e-02]], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 None)